Here is a partial implementation of the paper: <br />
"*A short note on parameter approximation for von Mises-Fisher distributions
And a fast implementation of Is(x)*"<br /> by Suvrit Sra

In [1]:
from scipy.special import iv as iv_scipy
from numpy import pi, power, exp, sqrt
from gmpy2 import mpfr, gamma, factorial, const_pi
from gmpy2 import exp as gmpexp
from gmpy2 import sqrt as gmpsqrt

In [2]:
def iv_s_paper(s, x, tau):
    """
    Computing Is(x) via truncated power-series (Paper version) 
    Beware : This has an algorithm issue !!
    """
    R, t1, t2 = 1., pow(x * exp(1.) / (2 * s), s), \
            1. \
            + 1. / (12 * s) \
            + 1. / (288 * s * s) \
            - 139. / (51840 * s * s * s)
    t1 = t1 * sqrt(s / (2 * pi)) / t2
    M, k, const_rat = 1. / s, 1, 0.25 * x * x
    convergence = False
    while not convergence:
        R = R* const_rat / (k * (s + k))
        M += R
        if R / M < tau:
            convergence = True
        k += 1
    return t1 * M

In [3]:
def iv_s(s, x, tau):
    """
    Computing Is(x) via truncated power-series (corrected version) 
    """
    t1, t2 = pow(x * exp(1.) / (2 * s), s), \
            1. \
            + 1. / (12 * s) \
            + 1. / (288 * s * s) \
            - 139. / (51840 * s * s * s)
    t1 *= sqrt(s / (2 * pi)) / t2
    k, tk, const_rat = 0, 1. / s, x * x / 4.    
    M = tk
    convergence = False
    while not convergence:
        R = const_rat / ((k + 1) * (s + k + 1))
        tk *= R
        k += 1
        M += tk
        if tk / M < tau:
            convergence = True
    return t1 * M

In [4]:
def iv_s_mpfr(s, x, tau):
    s, x = mpfr(s), mpfr(x)
    R, t1 = 1., pow(x * gmpexp(1.) / (2 * s), s)
    t2 = 1.+ 1. / (12 * s) + 1. / (288 * s * s) - 139. / (51840 * s * s * s)
    t1 = t1 * gmpsqrt(s / (2 * const_pi())) / t2
    k, tk, const_rat = 0, 1. / s, x * x / 4
    M = tk
    convergence = False
    while not convergence:
        R = const_rat / ((k + 1) * (s + k + 1))
        tk *= R
        M += tk
        if tk / M < tau:
            convergence = True
        k += 1
    return t1 * M

In [22]:
def iv_s_eq7 (s, x, order=1000000):
    s, x = mpfr(s), mpfr(x)
    t1 = pow(x / 2, s)
    t2 = mpfr(0)
    k = 0
    t2 = sum(pow(x * x / 4, k) / (gamma(k + s + 1) * factorial(k)) for k in range(order))
    return t1 * t2

def iv_s_eq8 (s, x):
    s, x = mpfr(s), mpfr(x)
    t1 = pow(x / 2, s) / gamma(s)
    t2 = mpfr(0)
    k = 0
    s_prod = s
    while k < 2:
        t2 += pow(x * x / 4, k) / (s_prod * factorial(k))   
        k += 1
        s_prod *= s + k
    return t1 * t2

### Verify correctness

In [23]:
tau = 1e-16
s = 200
v = 200
print '--', s , '--', v, '--'
print 'Scipy : ', iv_scipy(s, v)
print 'paper : ', iv_s_paper(s, v, tau), '(incorrect)'
print 'corrected : ',iv_s(s, v, tau)
print 'mpfr : ',iv_s_mpfr(s, v, tau)
print 'eq.7 : ',iv_s_eq7(s, v)
print 'eq.8 : ',iv_s_eq8(s, v)




-- 200 -- 200 --
Scipy :  4.54005913226e+44
paper :  9.08011826453e+46 (incorrect)
corrected :  4.54005913226e+44
mpfr :  4.5400591322630511e+44
eq.7 : 

KeyboardInterrupt: 

### Speed test

In [8]:
s = 5
for _ in range(10):
    %timeit iv_scipy(s,s)
    %timeit iv_s(s,s, tau)
    %timeit iv_s_mpfr(s,s, tau)
    %timeit iv_s_eq7(s,s)
    %timeit iv_s_eq8(s,s)


The slowest run took 8.87 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 2.26 µs per loop
The slowest run took 6.13 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 6.19 µs per loop
10000 loops, best of 3: 40.3 µs per loop
1000 loops, best of 3: 362 µs per loop
10000 loops, best of 3: 123 µs per loop
The slowest run took 9.77 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 2.05 µs per loop
The slowest run took 5.46 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 6.03 µs per loop
10000 loops, best of 3: 39.8 µs per loop
1000 loops, best of 3: 362 µs per loop
10000 loops, best of 3: 119 µs per loop
The slowest run took 9.39 times longer than the fastest. This could mean that an intermediate result is being cached 
10

In [ ]:
s  = 5e5

t_scipy = [